In [1]:
## Deepinsight Python
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import math
from scipy.spatial import distance
from statistics import mean
from PIL import Image
from matplotlib import cm
import os
import natsort


In [2]:
#list_dir = os.listdir('plots_515/umap_5_0.1')
#list_dir = os.listdir('plots_515/umap/data_finaldf')
#list_dir = natsort.natsorted(list_dir)

clinical = pd.read_csv("data/clinical_515.csv")
patient_id = clinical["sample"].values
status = clinical["vital status"].values

data = pd.read_csv("data/mrna_515.csv")
#data1 = pd.read_csv("data/mrna576.csv", sep = '\t')

In [ ]:
list_dir = os.listdir('plots_515/tsne_6_cosine/data_finaldf')
list_dir = natsort.natsorted(list_dir)

In [ ]:
for datafile in list_dir:
    df_subset = pd.read_csv("plots_515/tsne_6_cosine/data_finaldf/" + datafile) #tsne output
    #df_subset = pd.read_csv("plots_515/umap/data_finaldf/umap_5_10.0.csv") #umap output
    #df_subset = pd.read_csv("plots_515/tsne/data_finaldf/tsne_6___cosine.csv") # tsne_out
    df_subset = df_subset[['a', 'b']]
    print("Datafile:", datafile)

    def minimum_bounding_rectangle(points):
        """
        Find the smallest bounding rectangle for a set of points.
        Returns a set of points representing the corners of the bounding box.

        :param points: an nx2 matrix of coordinates
        :rval: an nx2 matrix of coordinates
        """
        from scipy.ndimage.interpolation import rotate
        pi2 = np.pi/2.

        # get the convex hull for the points
        hull_points = points[ConvexHull(points).vertices]

        # calculate edge angles
        edges = np.zeros((len(hull_points)-1, 2))
        edges = hull_points[1:] - hull_points[:-1]

        angles = np.zeros((len(edges)))
        angles = np.arctan2(edges[:, 1], edges[:, 0])

        angles = np.abs(np.mod(angles, pi2))
        angles = np.unique(angles)

        # find rotation matrices
        # XXX both work
        rotations = np.vstack([
            np.cos(angles),
            np.cos(angles-pi2),
            np.cos(angles+pi2),
            np.cos(angles)]).T
        rotations = rotations.reshape((-1, 2, 2))

        # apply rotations to the hull
        rot_points = np.dot(rotations, hull_points.T)

        # find the bounding points
        min_x = np.nanmin(rot_points[:, 0], axis=1)
        max_x = np.nanmax(rot_points[:, 0], axis=1)
        min_y = np.nanmin(rot_points[:, 1], axis=1)
        max_y = np.nanmax(rot_points[:, 1], axis=1)

        # find the box with the best area
        areas = (max_x - min_x) * (max_y - min_y)
        best_idx = np.argmin(areas)

        # return the best box
        x1 = max_x[best_idx]
        x2 = min_x[best_idx]
        y1 = max_y[best_idx]
        y2 = min_y[best_idx]
        r = rotations[best_idx]

        rval = np.zeros((4, 2))
        rval[0] = np.dot([x1, y2], r)
        rval[1] = np.dot([x2, y2], r)
        rval[2] = np.dot([x2, y1], r)
        rval[3] = np.dot([x1, y1], r)

        return rval

    mbr = minimum_bounding_rectangle(df_subset.values)

    # Calculate tilit of bounding box
    y2,y1,x2,x1 = mbr[0][1],mbr[1][1],mbr[0][0],mbr[1][0]
    theta = (y2-y1)/(x2-x1)
    angle  = math.degrees(np.arctan(theta))
    angle  = np.arctan(theta)
    r_matrix = np.asarray([[math.cos(angle),-1*math.sin(angle)],[math.sin(angle),math.cos(angle)]])

    zrect = np.matmul(mbr,r_matrix)  
    z = np.asarray(np.matmul(df_subset,r_matrix))
    rz_subset  = pd.DataFrame()
    rz_subset['rot-tsne-2d-one'] = z[:,0]
    rz_subset['rot-tsne-2d-two'] = z[:,1]
    rz_subset.to_csv('tsne_images/mrna515_tsne_6_cosine_100itr/mrna515_6_cosine_100itr_rotated.csv')

    z_dist = distance.cdist(z, z, 'euclidean')
    min_z_dist = z_dist[z_dist>0].min()

    rec_x_axis,rec_y_axis = abs(zrect[0][0] - zrect[1][0]),abs(zrect[1][1] - zrect[2][1])

    precision_old = math.sqrt(2)
    A = math.ceil((rec_x_axis*precision_old)/min_z_dist)
    B = math.ceil((rec_y_axis*precision_old)/min_z_dist)

    max_pix_size = 120
    if (max([A,B]) > max_pix_size):
        precision = (precision_old*max_pix_size/max([A,B]))
        A = math.ceil((rec_x_axis*precision)/min_z_dist)
        B = math.ceil((rec_y_axis*precision)/min_z_dist)

    x_coord,y_coord = rz_subset.iloc[:,0].values,rz_subset.iloc[:,1].values
    x_min,y_min,x_max,y_max = min(x_coord),min(y_coord),max(x_coord),max(x_coord)
    x_pixel = (1 + (A*(x_coord - x_min))/(x_max - x_min))
    y_pixel = (1 + (B*(y_coord - y_min))/(y_max - y_min))
    round_x_pixel = np.array([int(np.round(x)) for x in x_pixel])
    round_y_pixel = np.array([int(np.round(y)) for y in y_pixel])

    pix_subset  = pd.DataFrame()
    pix_subset['pix-tsne-2d-one'] = round_x_pixel
    pix_subset['pix-tsne-2d-two'] = round_y_pixel
    pix_subset.to_csv('tsne_images/mrna515_tsne_6_cosine_100itr/mrna51_6_cosine_100itr_pix_coord.csv')

    #function to find overlapping points
    def list_duplicates_of(seq,item):
        start_at = -1
        locs = []
        while True:
            try:
                loc = seq.index(item,start_at+1)
            except ValueError:
                break
            else:
                locs.append(loc)
                start_at = loc
        return locs

    #********************************For full image**************************
    pixel_set = []
    for _ in range(len(round_x_pixel)):
        pixel_set.append((round_x_pixel[_],round_y_pixel[_]))

    unique_coord = {}
    for i in range(len(pixel_set)):
        unique_coord[pixel_set[i]] = list_duplicates_of(pixel_set,pixel_set[i])

    clean_data = data.iloc[:,1:].T
    mmscaler = MinMaxScaler()
    #mmscaler.fit(clean_data)
    norm_data = pd.DataFrame(mmscaler.fit_transform(clean_data))
    sample_no = 0
    fvec = norm_data.iloc[sample_no,:]

    new_A = max(round_x_pixel)
    new_B = max(round_y_pixel)

    im_size =  max(new_A, new_B)

    for i in range(norm_data.shape[0]):
        sample_no = i
        imgM = np.zeros((im_size+1,im_size+1))
        fvec = norm_data.iloc[sample_no,:]
        for j in range(norm_data.shape[1]):
            imgM[round_x_pixel[j]-1,round_y_pixel[j]-1] = mean([fvec[k] for k in unique_coord.get((round_x_pixel[j],round_y_pixel[j]))])

        im = Image.fromarray(np.uint8(cm.Greys(imgM)*255))

        #np.save("UMAP_images/mrna515_0.1_5/data_px/" + datafile + "_" + patient_id[i], imgM)
        
        #plt.figure(figsize=(5,10))
        plt.axis('off')
        plt.imshow(imgM, cmap='BuPu',vmin=0, vmax=1)

        if status[i] == 0:
            np.save("tsne_images/mrna515_tsne_6_cosine_100itr/data_px/class_0/" + patient_id[i] + '_' + datafile, imgM)
            im.save('tsne_images/mrna515_tsne_6_cosine_100itr/class_0/' + patient_id[i] + '_' + datafile + '.png')
            #plt.savefig('tsne_images/mrna515_tsne_6_cosine/class_0/_' + patient_id[i] + '.png', bbox_inches='tight', format = "png", dpi=96)
            #plt.savefig('tsne_images/mrna515/class_0/_' + patient_id[i] + '.png', bbox_inches='tight', format = "png", dpi=96)
            
        else:
            np.save("tsne_images/mrna515_tsne_6_cosine_100itr/data_px/class_1/" + patient_id[i] + '_' + datafile, imgM)
            im.save('tsne_images/mrna515_tsne_6_cosine_100itr/class_1/' + patient_id[i] + '_' + datafile + '.png')
            #plt.savefig('tsne_images/mrna515_tsne_6_cosine/class_1/_' + patient_id[i] + '.png', bbox_inches='tight', format = "png", dpi=96)

        plt.close()

    print(datafile, "Done!")

In [21]:
#for cancer genes only image
color_idx = pd.read_csv("color_idx.csv")
final_df = pd.read_csv("plots_515/tsne/data_finaldf/tsne_6___cosine.csv")
#pix_subset

final_df = final_df.drop('c', axis =1)
final_df = pd.concat([final_df, color_idx], axis=1)
final_df.columns = ['a', 'b', 'c']

df2= pd.concat([pix_subset, final_df['c']], axis = 1)
df2 = df2.loc[df2['c']>=2] 
df3 = df2.drop(['c'], axis = 1)
# df3_a = df3['pix-tsne-2d-one'].values
# df3_b = df3['pix-tsne-2d-two'].values

# #final_df.columns = ['a', 'b', 'c']
# df = final_df.loc[final_df['c']>=2] 

# df2= pd.concat([pix_subset, df['c']], axis = 1)
# df2 = df2.loc[df2['c']>=2] 

# df3 = df2.drop(['c'], axis = 1)
df3_a = df3['pix-tsne-2d-one'].values
df3_b = df3['pix-tsne-2d-two'].values

In [49]:
#df3.reset_index(level=0, inplace=True)
idx_cancer_genes = df3['index'].values
np.save('analysis/idx_C_pix.npy', idx_cancer_genes)

In [47]:
idx_cancer_genes

array([  796,  1044,  1109,  1153,  1252,  1253,  1285,  1422,  1559,
        1560,  1577,  1623,  1636,  1714,  3220,  3221,  3222,  3224,
        3225,  3435,  3447,  3449,  3456,  3467,  3471,  3473,  3474,
        3654,  3655,  4289,  4366,  4370,  5072,  5106,  5241,  5678,
        5757,  6146,  6148,  6149,  6150,  6151,  6152,  6154,  6695,
        7087,  7097,  7434,  8179,  8180,  8301,  8392,  8528,  8834,
        8838, 10204, 10205, 10676, 10677, 10912, 11028, 11056, 11170,
       11174, 11301, 11977, 12795, 13832, 14202, 14471, 14477, 14589,
       15722, 16416, 16884, 16998, 17543, 18186, 19243, 19255, 19278])

In [31]:
#*******************for image with cancer genes********************
pixel_set_C = []
for _ in range(len(df3)):
    pixel_set_C.append((df3_a[_],df3_b[_]))

unique_coord_C = {}
for i in range(len(pixel_set_C)):
    unique_coord_C[pixel_set_C[i]] = list_duplicates_of(pixel_set_C,pixel_set_C[i])

# for i in range(norm_data.shape[0]):
#     sample_no = i
#     imgC = np.zeros((new_A+1,new_B+1))
#     #fvec = norm_data.iloc[sample_no,:]

#     #for j in range(norm_data.shape[1]):
#     for j in range(len(df3_a)):
#         imgC[df3_a[j], df3_b[j]] = 255

#         #if j%5 == 0:
#             #plt.imshow(imgC, cmap='BuPu')
#             #plt.savefig('image_' + str(j) + '.png', bbox_inches='tight', format = "png", dpi=96)
#         #print(df3_a[j], df3_b[j],j)

#     #np.save("UMAP_images/mrna515_0.1_5/data_px/" + datafile + "_" + patient_id[i], imgM)
#     #np.save("tsne_images/mrna515/data_px/_" + patient_id[i], imgC)
#     #plt.figure(figsize=(5,10))
#     #plt.axis('off')
#     #plt.imshow(imgC, cmap='BuPu',vmin=0, vmax=1)
#     plt.imshow(imgC, cmap='BuPu')
#     plt.savefig('image_final_cancer.png', bbox_inches='tight', format = "png", dpi=96)

#     # if status[i] == 0:
#     #     plt.savefig('tsne_images/mrna515/class_0/_' + patient_id[i] + '.png', bbox_inches='tight', format = "png", dpi=96)
#     #     #plt.savefig('tsne_images/mrna515/class_0/_' + patient_id[i] + '.png', bbox_inches='tight', format = "png", dpi=96)
        
#     # else:
#     #     plt.savefig('tsne_images/mrna515/class_1/_' + patient_id[i] + '.png', bbox_inches='tight', format = "png", dpi=96)

#     # plt.close()

# # print(patient_id[i], "Done!")

In [41]:
#To extract location of cancer genes and all the genes in 122x122 pixelsize image
len(unique_coord_C)
np.save('analysis/genlist_pix.npy', unique_coord)
np.save('analysis/genlist_C_pix.npy', unique_coord_C)

In [3]:
#df_subset = pd.read_csv("plots_515/tsne_6_cosine/data_finaldf/" + datafile) #tsne output
df_subset = pd.read_csv("plots_515/tsne_6_cosine/data_finaldf/tsne_6___cosine.csv") #umap output
#df_subset = pd.read_csv("plots_515/tsne/data_finaldf/tsne_6___cosine.csv") # tsne_out
df_subset = df_subset[['a', 'b']]
#print("Datafile:", datafile)

def minimum_bounding_rectangle(points):
    """
    Find the smallest bounding rectangle for a set of points.
    Returns a set of points representing the corners of the bounding box.

    :param points: an nx2 matrix of coordinates
    :rval: an nx2 matrix of coordinates
    """
    from scipy.ndimage.interpolation import rotate
    pi2 = np.pi/2.

    # get the convex hull for the points
    hull_points = points[ConvexHull(points).vertices]

    # calculate edge angles
    edges = np.zeros((len(hull_points)-1, 2))
    edges = hull_points[1:] - hull_points[:-1]

    angles = np.zeros((len(edges)))
    angles = np.arctan2(edges[:, 1], edges[:, 0])

    angles = np.abs(np.mod(angles, pi2))
    angles = np.unique(angles)

    # find rotation matrices
    # XXX both work
    rotations = np.vstack([
        np.cos(angles),
        np.cos(angles-pi2),
        np.cos(angles+pi2),
        np.cos(angles)]).T
    rotations = rotations.reshape((-1, 2, 2))

    # apply rotations to the hull
    rot_points = np.dot(rotations, hull_points.T)

    # find the bounding points
    min_x = np.nanmin(rot_points[:, 0], axis=1)
    max_x = np.nanmax(rot_points[:, 0], axis=1)
    min_y = np.nanmin(rot_points[:, 1], axis=1)
    max_y = np.nanmax(rot_points[:, 1], axis=1)

    # find the box with the best area
    areas = (max_x - min_x) * (max_y - min_y)
    best_idx = np.argmin(areas)

    # return the best box
    x1 = max_x[best_idx]
    x2 = min_x[best_idx]
    y1 = max_y[best_idx]
    y2 = min_y[best_idx]
    r = rotations[best_idx]

    rval = np.zeros((4, 2))
    rval[0] = np.dot([x1, y2], r)
    rval[1] = np.dot([x2, y2], r)
    rval[2] = np.dot([x2, y1], r)
    rval[3] = np.dot([x1, y1], r)

    return rval

mbr = minimum_bounding_rectangle(df_subset.values)

# Calculate tilit of bounding box
y2,y1,x2,x1 = mbr[0][1],mbr[1][1],mbr[0][0],mbr[1][0]
theta = (y2-y1)/(x2-x1)
angle  = math.degrees(np.arctan(theta))
angle  = np.arctan(theta)
r_matrix = np.asarray([[math.cos(angle),-1*math.sin(angle)],[math.sin(angle),math.cos(angle)]])

zrect = np.matmul(mbr,r_matrix)  
z = np.asarray(np.matmul(df_subset,r_matrix))
rz_subset  = pd.DataFrame()
rz_subset['rot-tsne-2d-one'] = z[:,0]
rz_subset['rot-tsne-2d-two'] = z[:,1]
#rz_subset.to_csv('tsne_images/mrna515_tsne_6_cosine/mrna515_6_cosine_rotated.csv')

z_dist = distance.cdist(z, z, 'euclidean')
min_z_dist = z_dist[z_dist>0].min()

rec_x_axis,rec_y_axis = abs(zrect[0][0] - zrect[1][0]),abs(zrect[1][1] - zrect[2][1])

precision_old = math.sqrt(2)
A = math.ceil((rec_x_axis*precision_old)/min_z_dist)
B = math.ceil((rec_y_axis*precision_old)/min_z_dist)

max_pix_size = 120
if (max([A,B]) > max_pix_size):
    precision = (precision_old*max_pix_size/max([A,B]))
    A = math.ceil((rec_x_axis*precision)/min_z_dist)
    B = math.ceil((rec_y_axis*precision)/min_z_dist)

x_coord,y_coord = rz_subset.iloc[:,0].values,rz_subset.iloc[:,1].values
x_min,y_min,x_max,y_max = min(x_coord),min(y_coord),max(x_coord),max(x_coord)
x_pixel = (1 + (A*(x_coord - x_min))/(x_max - x_min))
y_pixel = (1 + (B*(y_coord - y_min))/(y_max - y_min))
round_x_pixel = np.array([int(np.round(x)) for x in x_pixel])
round_y_pixel = np.array([int(np.round(y)) for y in y_pixel])

pix_subset  = pd.DataFrame()
pix_subset['pix-tsne-2d-one'] = round_x_pixel
pix_subset['pix-tsne-2d-two'] = round_y_pixel
#pix_subset.to_csv('tsne_images/mrna515_tsne_6_cosine/mrna515_6_cosine_pix_coord.csv')

#function to find overlapping points
def list_duplicates_of(seq,item):
    start_at = -1
    locs = []
    while True:
        try:
            loc = seq.index(item,start_at+1)
        except ValueError:
            break
        else:
            locs.append(loc)
            start_at = loc
    return locs

#********************************For full image**************************
pixel_set = []
for _ in range(len(round_x_pixel)):
    pixel_set.append((round_x_pixel[_],round_y_pixel[_]))

unique_coord = {}
for i in range(len(pixel_set)):
    unique_coord[pixel_set[i]] = list_duplicates_of(pixel_set,pixel_set[i])

clean_data = data.iloc[:,1:].T
mmscaler = MinMaxScaler()
#mmscaler.fit(clean_data)
norm_data = pd.DataFrame(mmscaler.fit_transform(clean_data))
sample_no = 0
fvec = norm_data.iloc[sample_no,:]

new_A = max(round_x_pixel)
new_B = max(round_y_pixel)

im_size =  max(new_A, new_B)

# for i in range(norm_data.shape[0]):
#     sample_no = i
#     imgM = np.zeros((im_size+1,im_size+1))
#     fvec = norm_data.iloc[sample_no,:]
#     for j in range(norm_data.shape[1]):
#         imgM[round_x_pixel[j]-1,round_y_pixel[j]-1] = mean([fvec[k] for k in unique_coord.get((round_x_pixel[j],round_y_pixel[j]))])

#     im = Image.fromarray(np.uint8(cm.Greys(imgM)*255))

#     #np.save("UMAP_images/mrna515_0.1_5/data_px/" + datafile + "_" + patient_id[i], imgM)
    
#     #plt.figure(figsize=(5,10))
#     plt.axis('off')
#     plt.imshow(imgM, cmap='BuPu',vmin=0, vmax=1)

#     if status[i] == 0:
#         np.save("tsne_images/mrna515_tsne_6_cosine/data_px/class_0/" + patient_id[i], imgM)
#         im.save('tsne_images/mrna515_tsne_6_cosine/class_0/' + patient_id[i] + '.png')
#         #plt.savefig('tsne_images/mrna515_tsne_6_cosine/class_0/_' + patient_id[i] + '.png', bbox_inches='tight', format = "png", dpi=96)
#         #plt.savefig('tsne_images/mrna515/class_0/_' + patient_id[i] + '.png', bbox_inches='tight', format = "png", dpi=96)
        
#     else:
#         np.save("tsne_images/mrna515_tsne_6_cosine/data_px/class_1/" + patient_id[i], imgM)
#         im.save('tsne_images/mrna515_tsne_6_cosine/class_1/' + patient_id[i] + '.png')
#         #plt.savefig('tsne_images/mrna515_tsne_6_cosine/class_1/_' + patient_id[i] + '.png', bbox_inches='tight', format = "png", dpi=96)

#     plt.close()

# print("Done!")

In [ ]:
unique_coord
# Save
#np.save('genlist_pix.npy', unique_coord)

In [ ]:
import imageio
import os
images = []
for filename in os.listdir("UMAP_results"):

    if filename.endswith(('.png')):
        images.append(imageio.imread("UMAP_results/" + filename))
imageio.mimsave('movie.gif', images)